In [1]:
import pandas as pd
import numpy as np
import os
import neat
import pickle
import ta
from collections import deque
import random
import time
from statistics import mean
from sklearn import preprocessing

#hide warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
sequence_length = 30
trade_list = []
current_stock = ""

In [3]:
#reading all csv from data folder and combine them all
def load_data():

    dfs = []

    for item in os.listdir('data'):
        df = pd.read_csv(f'data/{item}',
                        header=None,
                        names=['stock code','Date','Open','High','Low','Close','Volume','Netforeign'])
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df.dropna(inplace=True)
        df.set_index('Date',inplace=True)

        #sort values by date
        df.sort_values('Date',inplace=True)
        dfs.append(df)

    main_df = pd.concat(dfs)
    main_df.tail()
    
    ##################################################################################################
    #read tradeble stocks
    tradable = pd.read_csv('tradable.csv')
    
    #creating a new df of tradable stock
    tradable_stock_df = main_df[main_df['stock code'].isin(tradable['stock'])]
    tradable_stock_df.head()
    
    tradable_stock_list = tradable_stock_df['stock code'].unique()
    tradable_stock_list.sort()

    print(tradable_stock_list,len(tradable_stock_list))
    print("\n\n")
    
    #group by tradable stock
    tradable_stock_df=tradable_stock_df.groupby('stock code')
    
    return tradable_stock_df

In [4]:
def process_data(data):
    data.drop("Netforeign",1,inplace=True) #drop netforeign
    data.drop("stock code",1,inplace=True) #drop stock code
    data['Volume'].replace(0,1,inplace=True) #replace 0 value volume with 1
    data.interpolate(inplace=True)
    
    #adding technical indicators
    data = ta.add_all_ta_features(data,
                              open="Open",
                              high="High",
                              low="Low",
                              close="Close",
                              volume="Volume",
                              fillna=False)
#     print(data.tail())
#     data.to_csv("dd.csv",index=False)
    
    col_list = ["Close","Volume",
            "momentum_rsi","momentum_wr","momentum_ao","momentum_stoch","momentum_stoch_signal",
            "trend_trix","trend_vortex_ind_pos","trend_vortex_ind_neg",
            "trend_vortex_ind_diff","trend_macd","trend_macd_signal","trend_macd_diff",
           "volatility_atr"]
    
    data = data[col_list]
    
    #OPEN , HIGH, LOW, CLOSE, VOLUME
    data['Close'] = data['Close']
    data['Volume'] = data['Volume']
    data['momentum_rsi'] = data['momentum_rsi']
    data['momentum_wr'] = data['momentum_wr']
    data['momentum_stoch'] = data['momentum_stoch']
    data['momentum_stoch_signal'] = data['momentum_stoch_signal']
    data['Pure_Close'] = data['Close']
    
    data.dropna(inplace=True)
    
    for col in data.columns[:-1]:
        data[col] = preprocessing.scale(data[col].values)
    
    return data

In [5]:
def deque_sequence(data):
    global sequence_length
    #initialize deque .. sequence of
    #number of sequence by open ,high ,low ,close++
    sequence = deque(maxlen=sequence_length)
    
    #if sequence_length = 6 .. it looks like this.. a 6 by 5 matrix
    for _ in range(sequence_length):
        sequence.append([0 for _ in data.columns[:-1]])
        
        """ 6(sequence) x 5(open,high,low,close)
                        0, 0, 0, 0, 0
                        0, 0, 0, 0, 0
                        0, 0, 0, 0, 0
                        0, 0, 0, 0, 0
                        0, 0, 0, 0, 0 """
#     print(sequence)
    return sequence

In [6]:
def get_action(data,net):
    #1stock
    global current_stock
    position_days = 0
    current_position = 0
    stock_reward = 0
    wins = 0
    loss = 0
    #initialize deque sequence
    sequence = deque_sequence(data)

    """ 6(sequence) x 5(open,high,low,close)
            0, 0, 0, 0, 0
            0, 0, 0, 0, 0
            0, 0, 0, 0, 0
            0, 0, 0, 0, 0
            0, 0, 0, 0, 0 """
    
    i = 0
    #FEEDING THE NEURAL NET
    for vals in data.values:
#         print(vals[:-1])
#         print(vals[-1])  ##pure_close
        current_price = vals[-1]
        #append the values of data (open,high,low,close) to deque sequence
        sequence.append(vals[:-1])
        
        #convert deque function to a numpy array
        x = np.array(sequence)

        #flatten features
        x = x.flatten()   

#         #append positon_change and position days ... more feature
#         x = np.append(x,[position_change,position_days])
        
#         #feed features to neural network
        output = net.activate(x)

#         #action recomended by the neural network 
        action = np.argmax(output, axis=0)      
        
#         print(action)
        current_position ,position_days ,reward = do_action(data.index[i],current_price,action,position_days,current_position)
        stock_reward += reward
        
        i+=1
        if reward == 1:
            wins += 1
        elif reward == -1:
            loss += 1
    
    print(f"wins:{wins} loss:{loss}")
    current_stock = ""
    
    return stock_reward

In [7]:
def do_action(date,current_price,action,position_days,current_position):
    global current_stock
    global total_profit
    global trade_list
    reward = 0
    position_change = (current_price-current_position)/current_position

    
#     """if action is BUY and has no position"""
    if action == 1 and position_days == 0 :
        position_days = 1
        current_position = current_price
        trade_list.append([date,current_stock,current_price,1])
        
        
    
#     """if action is BUY and has position"""
    elif action == 1 and position_days > 0 :
        position_days += 1
        
        #SELL if position is 60 days older or -10%
        if position_days >= 45 or position_change < -5/100:
            #SELL
            trade_list.append([date,current_stock,current_price,2])
            #check trade if win or loss
            if position_change >  1.19/100:
                reward = 1
                total_profit += position_change*100
#                 print(f"profit:{position_change*100} days:{position_days}")
                
            else:
                reward = -1
                total_profit += position_change*100
#                 print(f"profit:{position_change*100} days:{position_days}")
                
            
            #RESET
            position_days = 0
            
#     """if action is SELL and has no position"""
    elif action == 2 and position_days == 0 :
        pass
    
#     """if action is SELL and has position"""
    elif action == 2 and position_days > 0 :
        position_days += 1
        trade_list.append([date,current_stock,current_price,2])
        if position_change > 1.19/100:
            reward = 1
            total_profit += position_change*100
#             print(f"profit:{position_change*100} days:{position_days}")
            
        else:
            reward = -1
            total_profit += position_change*100
#             print(f"profit:{position_change*100} days:{position_days}")
            
        
        #RESET
        position_days = 0
        
#     """if action is hold and has no position"""
    elif action == 0 and position_days == 0 :
        pass
    
#     """if action is hold and has position"""
    elif action == 0 and position_days > 0 :
        position_days += 1
        
        #sell if position is 60 days older
        if position_days >= 45 or position_change < -5/100:
            #SELL
            trade_list.append([date,current_stock,current_price,2])
            #check trade if win or loss
            if position_change > 1.19/100 :
                reward = 1
                total_profit += position_change*100
#                 print(f"profit:{position_change*100} days:{position_days}")
            else:
                reward = -1
                total_profit += position_change*100
#                 print(f"profit:{position_change*100} days:{position_days}")
            
            #RESET
            position_days = 0

        
        
            
    return current_position,position_days,reward

In [8]:
stock = "FLI"
total_profit = 0

In [9]:
#loading data
stock_data = load_data()

['ABA' 'ABG' 'AC' 'ACEN' 'ACEX' 'AEV' 'AGI' 'ALCO' 'ALI' 'ALLHC' 'ANI'
 'AP' 'APC' 'APX' 'AXLM' 'BCOR' 'BDO' 'BLOOM' 'BPI' 'BRN' 'C' 'CEB' 'CHP'
 'CIC' 'CNPF' 'COSCO' 'CPG' 'CPM' 'DD' 'DITO' 'DMC' 'DNL' 'EAGLE' 'EEI'
 'EMP' 'EW' 'FB' 'FGEN' 'FLI' 'FNI' 'FOOD' 'FRUIT' 'GERI' 'GLO' 'GTCAP'
 'HLCM' 'HOME' 'HOUSE' 'ICT' 'IDC' 'IMI' 'ION' 'IRC' 'JFC' 'JGS' 'KPPI'
 'LPZ' 'LR' 'LTG' 'MAC' 'MAH' 'MAXS' 'MBT' 'MEG' 'MER' 'MHC' 'MPI' 'MRC'
 'MRSGI' 'MWC' 'MWIDE' 'NIKL' 'NOW' 'ORE' 'PCOR' 'PGOLD' 'PHA' 'PHES'
 'PIP' 'PIZZA' 'PLC' 'PNB' 'PNX' 'PRMX' 'PX' 'PXP' 'RCB' 'RLC' 'RRHI'
 'RWM' 'SCC' 'SECB' 'SHLPH' 'SLI' 'SM' 'SMC' 'SMPH' 'SSI' 'STI' 'STR'
 'TBGI' 'TECH' 'TEL' 'TUGS' 'UBP' 'URC' 'VITA' 'VLL' 'VUL' 'WIN' 'WLCON'
 'WPI' 'X' '^PSEi'] 114





In [10]:
def run(config_file):
    #globals
    global stock_data
    global current_stock
    global trade_list
#     global stock
    total_reward = 0
    global total_profit
    p = 0
    
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)
    
    winner = pickle.load(open("nets/winner_net0.pickle","rb"))
    
    winner_net = neat.nn.RecurrentNetwork.create(winner, config)
    
    #testing stocks
#     testing_stocks = ["IDC","AXLM","TECH","WLCON","GREEN","MAXS","JFC","CHP","FRUIT","PGOLD","MAH","DITO","MWIDE"]
    
    
    for stock,data in stock_data:
#     for stock in testing_stocks:
        current_stock = stock
        data = stock_data.get_group(stock)
        data.sort_values('Date',inplace=True)
        print(len(data),stock,data.index[-1])
        data = process_data(data)
        # yy-mm-dd
#         data = data.loc['2015-01-01':'2019-11-05']
#         data = data.loc[:'2018-11-05']
        data = data.loc[:]
#         data = data.loc['2020-02-01':]
        if len(data) > 30:
            stock_reward = get_action(data,winner_net)
            total_reward += stock_reward

        print(f"{total_profit}")
        p += total_profit
        total_profit = 0
        
    print(p)
    trade_list = pd.DataFrame(trade_list,columns=['Date', 'Stock', 'Price','Action'])
    trade_list.to_csv("trade_list.csv",index=False)

In [11]:
if __name__ == '__main__':
    # Determine path to configuration file. This path manipulation is
    # here so that the script will run successfully regardless of the
    # current working directory.
    local_dir = os.path.dirname('__file__')
    config_path = os.path.join(local_dir, 'config-feedforward.txt')
    run(config_path)

3328 ABA 2020-10-27 00:00:00
wins:104 loss:128
10902.053467021638
2042 ABG 2020-10-27 00:00:00
wins:49 loss:83
-31.176079266942732
3460 AC 2020-10-27 00:00:00
wins:90 loss:108
-20.396996863974454
953 ACEN 2020-08-13 00:00:00
wins:17 loss:28
-37.75254976029841
775 ACEX 2020-10-27 00:00:00
wins:27 loss:24
110.11283049108665
3447 AEV 2020-10-27 00:00:00
wins:105 loss:71
222.9807211559499
3329 AGI 2020-10-27 00:00:00
wins:95 loss:114
-51.31421585820087
2398 ALCO 2020-10-27 00:00:00
wins:89 loss:82
317.7173730046859
3460 ALI 2020-10-27 00:00:00
wins:114 loss:90
171.0162560771592
2751 ALLHC 2020-10-27 00:00:00
wins:95 loss:95
398.68575551839456
2679 ANI 2020-10-27 00:00:00
wins:82 loss:90
-10.396159799676482
3216 AP 2020-10-27 00:00:00
wins:112 loss:71
293.44444463210607
3372 APC 2020-10-27 00:00:00
wins:122 loss:121
125.60728427366217
3155 APX 2020-10-27 00:00:00
wins:109 loss:123
234.25429271437503
259 AXLM 2020-10-27 00:00:00
wins:8 loss:11
-15.664086485110685
1250 BCOR 2020-10-27 00:00:0